In [1]:
from malaysian_sft import accept
from collections import defaultdict
import pandas as pd
import os
import json
import random
import re

In [2]:
from huggingface_hub import hf_hub_download
import fasttext

filename = hf_hub_download(
    repo_id="mesolitica/fasttext-language-detection-bahasa-en", 
    filename="fasttext.ftz"
)
lang_model = fasttext.load_model(filename)

In [3]:
from tqdm import tqdm

def generate_and_tokenize_prompt(row, validate = True):
    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except BaseException:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human)
            outputs.append(bot)
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    chat = []
    if row['prompt_input'] is not None and len(row['prompt_input']):
        chat.append({'role': 'system', 'content': row['prompt_input'].strip()})
    for input, output in zip(inputs, outputs):
        chat.extend([
            {'role': 'user', 'content': input.strip()},
            {'role': 'assistant', 'content': output.strip()},
        ])
    if validate:
        for c in chat:
            if not accept(c['content']):
                return
    return chat

In [4]:
df = pd.read_parquet('google_translate_camel_ai-00000-of-00001.parquet')
df.head()

,prompt_input,input,output
0,None,Cari bucu bagi fungsi nilai mutlak f(x) = |2x ...,Untuk mencari bucu bagi fungsi nilai mutlak f(...
1,None,Biarkan $f:\mathbb{R}^2 \to \mathbb{R}$ ditakr...,"Untuk mencari titik genting bagi fungsi $f(x, ..."
2,None,"Sudah tentu, berikut ialah masalah berangka un...",Untuk mencari kecerunan garis tangen kepada le...
3,None,Katakan kita mempunyai dua set A dan B di mana...,"Untuk mencari persilangan set A dan B, kita me..."
4,None,Terdapat 10 bola merah yang sama dan 5 bola bi...,"Untuk menyelesaikan masalah ini, kita boleh me..."


In [5]:
camel_instruct = []
for i in range(len(df)):
    camel_instruct.append([
        {'role': 'user', 'content': df.iloc[i]['input']},
        {'role': 'assistant', 'content': df.iloc[i]['output']}
    ])

In [6]:
camel_instruct[0]

[{'role': 'user',
  'content': 'Cari bucu bagi fungsi nilai mutlak f(x) = |2x - 5|.'},
 {'role': 'assistant',
  'content': 'Untuk mencari bucu bagi fungsi nilai mutlak f(x) = |2x - 5|, kita perlu mencari koordinat-x bagi puncak tersebut. Puncak berlaku pada titik di mana ungkapan di dalam nilai mutlak adalah sama dengan sifar.\n\n2x - 5 = 0\n\nSekarang, selesaikan untuk x:\n\n2x = 5\nx = 5/2\n\nKoordinat-x bagi bucu ialah 5/2. Sekarang, kita perlu mencari koordinat-y yang sepadan dengan memasukkan semula koordinat-x ke dalam fungsi:\n\nf(5/2) = |2(5/2) - 5|\n\nf(5/2) = |5 - 5|\n\nf(5/2) = 0\n\nJadi, koordinat y bagi bucu ialah 0. Oleh itu, bucu bagi fungsi nilai mutlak f(x) = |2x - 5| ialah (5/2, 0).'}]

In [7]:
df = pd.read_parquet('peribahasa_instructions-00000-of-00001.parquet')
df

,prompt_input,input,output
0,None,"apa mksud peribahasa, ada aku dipandang hadap,...","Kasih sayang hanya waktu berhadapan saja, sete..."
1,None,"apa mksud peribahasa, ada angin",Orang yang berperangai luar biasa pada masa te...
2,None,"apa mksud peribahasa, ada angin, ada pohonnya",Setiap kejadian pasti ada sebab-sebabnya (asal...
3,None,"ada angin, ada pokoknya, mksud peribahasa",Setiap kejadian pasti ada mulanya.
4,None,"ada bakat, mksud peribahasa",Berkebolehan dalam sesuatu lapangan seperti me...
...,...,...,...
8316,None,"apa mksud peribahasa, zaman meleset",Zaman serba susah
8317,None,"zaman Tok Naduk, mksud peribahasa",Zaman awal bagi Pulau Pinang diterokai oleh Pi...
8318,None,"zaman tuk nadur berkajang kain, mksud peribahasa",Zaman yang lampau; sudah lama sekali
8319,None,"apa mksud peribahasa, zat makanan",Zat-zat yang diperoleh dari makanan manusia


In [8]:
peribahasa_instruct = []

for i in range(len(df)):
    if len(df.iloc[i]['output']) > 4 and len(df.iloc[i]['input']) > 4:
        peribahasa_instruct.append([
            {'role': 'user', 'content': df.iloc[i]['input']}, 
            {'role': 'assistant', 'content': df.iloc[i]['output']}
        ])
        
len(peribahasa_instruct)

8321

In [9]:
jawi_translation_instruct = []
files = ['en-jawi.parquet', 'ms-jawi.parquet', 'jawi-ms.parquet', 'jawi-en.parquet']
for f in files:
    df = pd.read_parquet(f).sample(5000)
    for i in range(len(df)):
        if len(df.iloc[i]['output']) > 4 and len(df.iloc[i]['input']) > 4:
            jawi_translation_instruct.append([
                {'role': 'user', 'content': df.iloc[i]['input']}, 
                {'role': 'assistant', 'content': df.iloc[i]['output']}
            ])
    

In [10]:
len(jawi_translation_instruct)

20000

In [11]:
!ls *force*.parquet

prepared-force-english.parquet	    prepared-force-malay.parquet
prepared-force-indonesian.parquet   prepared-force-mandarin.parquet
prepared-force-jawi.parquet	    prepared-force-tamil.parquet
prepared-force-json-format.parquet


In [12]:
df = pd.read_parquet('prepared-force-malay.parquet')
df

,prompt_input,input,output
0,None,sentiasa respond dalam bahasa melayu\n\nIn a g...,"Pertama, mari kita mulakan dengan melaksanakan..."
1,None,u always reply in malay\n\nThere is a mysterio...,"1. Apabila `i` sama dengan 0, terdapat 3 kedai..."
2,None,always reply in malay\n\nYou are given a direc...,Berikut adalah penyelesaian yang mungkin dalam...
3,None,awak sentiasa reply in malay\n\nIn a blockchai...,Berikut adalah penyelesaian C++ untuk masalah ...
4,None,sentiasa respond dalam bahasa melayu\n\nThere ...,"Pertama, ubah struktur `cli_args` untuk memasu..."
...,...,...,...
625075,None,<manusia>: sentiasa respond dalam bahasa melay...,None
625076,None,<manusia>: awak sentiasa reply in malay\n\nWhy...,None
625077,None,<manusia>: awak sentiasa reply in malay\n\nWhy...,None
625078,None,<manusia>: sentiasa respond dalam bahasa melay...,None


In [13]:
files = {
    'prepared-force-malay.parquet': 50000,
    'prepared-force-jawi.parquet': 50000, 
    'prepared-force-tamil.parquet': 10000, 
    'prepared-force-english.parquet': 5000, 
    'prepared-force-mandarin.parquet': 10000, 
    'prepared-force-indonesian.parquet': 10000,
}

In [14]:
for f, v in files.items():
    filename = f'{f}.filtered'
    if os.path.exists(filename):
        continue
    
    rs, codes = [], []
    df = pd.read_parquet(f)
    for i in tqdm(range(len(df))):
        row = generate_and_tokenize_prompt(df.iloc[i].to_dict(), validate = True)
        if row is None:
            continue
        code = False
        for r in row:
            if '```' in r['content']:
                code = True
                break
        
        if code:
            codes.append(row)
        else:
            rs.append(row)
            
    with open(filename, 'w') as fopen:
        json.dump({'rs': rs, 'codes': codes}, fopen)

In [15]:
force = defaultdict(list)

templates = [
    '{system}\n\n{text}',
    '{text}, {system}',
    '{system}, {text}'
]

for f, v in files.items():
    filename = f'{f}.filtered'
    if not os.path.exists(filename):
        continue
    
    with open(filename) as fopen:
        d = json.load(fopen)
    
    samples = random.sample(d['rs'], min(v, len(d['rs'])))
    s = []
    for i in range(len(samples)):
        splitted = samples[i][0]['content'].split('\n\n')
        if len(splitted[0]) < 3:
            continue
        samples[i][0]['content'] = splitted[1]
        if random.random() > 0.3:
            samples[i].insert(0, {'role': 'system', 'content': splitted[0]})
        else:
            samples[i][0]['content'] = random.choice(templates).format(system = splitted[0], text = splitted[1])
            samples[i] = samples[i][:2]
            
        s.append(samples[i])
        
    force[f].extend(s)
    
    try:
        s = []
        samples = random.sample(d['codes'], min(v // 5, len(d['codes'])))
        for i in range(len(samples)):
            splitted = samples[i][0]['content'].split('\n\n')
            if len(splitted[0]) < 3:
                continue
            samples[i][0]['content'] = splitted[1]
            if random.random() > 0.3:
                samples[i].insert(0, {'role': 'system', 'content': splitted[0]})
            else:
                samples[i][0]['content'] = random.choice(templates).format(system = splitted[0], text = splitted[1])
                samples[i] = samples[i][:2]
            
            s.append(samples[i])

        force[f].extend(s)
    except:
        pass

len(force)

6

In [16]:
force.keys()

dict_keys(['prepared-force-malay.parquet', 'prepared-force-jawi.parquet', 'prepared-force-tamil.parquet', 'prepared-force-english.parquet', 'prepared-force-mandarin.parquet', 'prepared-force-indonesian.parquet'])

In [17]:
len(force['prepared-force-malay.parquet'])

60000

In [18]:
jawi = []
with open('synthetic-jawi-conversation.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)

        try:
            s = l[1]['content_ms'].split()
            if len(s) < 250:
                continue

            if len(s) > 1000:
                continue

            if not accept(l[1]['content_ms']):
                continue      

            jawi.append(l[:2])

        except Exception as e:
            pass

system = [
    'you are a chatbot that always reply in jawi',
    'you are an assistant that always reply in jawi',
    'awak adalah chatbot yang sentiasa respond dalam jawi',
    'awak adalah pembantu AI yang reply in jawi',
]

system2 = [
    'respond in jawi, {text}',
    '{text}, reply in jawi',
    'reply in jawi\n\n{text}',
    '{text}\n\nrespond dlm jawi',
]

j = [j for j in jawi if j[0]['role'] != 'context']

j = sorted(j, key = lambda x: len(x[1]['content_ms']), reverse = True)[:50000]

jawi_instructions = []
for j_ in j:
    if random.randint(0, 1):
        l = random.choice(system2).format(text = j_[0]['content_ms'])
        jawi_instructions.append([
            {'role': 'user', 'content': l},
            {'role': 'assistant', 'content': j_[1]['content_jawi']}
        ])
    else:
        s = random.choice(system)
        jawi_instructions.append([
            {'role': 'system', 'content': s},
            {'role': 'user', 'content': j_[0]['content_ms']},
            {'role': 'assistant', 'content': j_[1]['content_jawi']}
        ])

509375it [06:52, 1235.17it/s] 


In [19]:
len(jawi_instructions)

50000

In [20]:
code = []

df = pd.read_parquet('chatgpt4_code_instruct-00000-of-00001.parquet')
for i in tqdm(range(len(df))):
    l = str(df.iloc[i].to_dict())
    if 'anda berikan' in l.lower():
        continue
    if not accept(df.iloc[i]['input'], skip_indon = False):
        continue
    if not accept(df.iloc[i]['output'], min_len = 1):
        continue
    d = [
        {'role': 'user', 'content': df.iloc[i]['input']},
        {'role': 'assistant', 'content': df.iloc[i]['output']}
    ]
    code.append(d)
    
df = pd.read_parquet('prepared-mixtral-magicoder.parquet')
for i in tqdm(range(len(df))):
    d = [
        {'role': 'user', 'content': df.iloc[i]['input']},
        {'role': 'assistant', 'content': df.iloc[i]['output']}
    ]
    code.append(d)
    
len(code)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 87556/87556 [00:02<00:00, 39315.41it/s]


114853

In [21]:
df = pd.read_parquet('malaysian_ultrachat-00000-of-00001.parquet')
df.head()

,prompt_input,input,output
0,None,<manusia>: English \n\n\n\nconcepts of patria...,None
1,None,<manusia>: © 2022 Sains Insani \n\n\n\n\n\n\n...,None
2,None,<manusia>: a ini. Sumbang saran dilakukan supa...,None
3,None,<manusia>: 10: 3-4 (2018) 27–33 | www.sainshum...,None
4,None,<manusia>: they lost their freedom and rights...,None


In [22]:
filter_ultrachat = []
for i in tqdm(range(len(df))):
    l = generate_and_tokenize_prompt(df.iloc[i].to_dict(), validate = False)
    valid = True
    for no, l_ in enumerate(l):
        left = re.sub('[^a-z ]+', ' ', l_['content'][:20].lower())
        left = re.sub(r'[ ]+', ' ', left).strip()
        right = re.sub('[^a-z ]+', ' ', l[no - 1]['content'][:20].lower())
        right = re.sub(r'[ ]+', ' ', right).strip()
        if left[:10] == right[:10]:
            valid = False
            break
    if not valid:
        continue
    l_ = json.dumps(l).lower()
    if 'dalam konteks di' in l_:
        continue
    if 'terjemah' in l_:
        continue
    if 'artikel itu' in l_:
        continue
    filter_ultrachat.append(l)
    
len(filter_ultrachat)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 149054/149054 [00:13<00:00, 10797.81it/s]


140432

In [23]:
filter_ultrachat[-1]

[{'role': 'user',
  'content': 'menimbangkan masalah ini dengan mengadakan penggredan sekolah yang wajar mengadakan kelas Tingkatan Enam. Bagi melahirkan seorang doktor yang terlatih, seorang pelajar perubatan harus melalui enam tahun pengajian di universiti, diikuti dengan setahun bekerja sebagai doktor pelatih atau housemanship dengan izin. Dalam tempoh inilah, doktor pelatih didedahkan dengan segala aspek kerja-kerja di hospital dan merawat jenis-jenis penyakit dan pesakit. Malangnya di negara ini, pengurusan sistem housemanship dengan izin, dilaporkan tidaklah menyenangkan. Ramai daripada doktor-doktor pelatih disebabkan dibebani dengan kerja-kerja yang berlebihan sehingga terpaksa bekerja 36 jam terus- menerus tanpa sebarang rehat. Perkara ini berlaku di hospital-hospital di seluruh negara dari dahulu hinggalah ke hari ini. Ini dapat disahkan oleh doktor-doktor senior yang masih ada sekarang. Sistem penderaan yang dikatakan ini bermula di kalangan pelatih baru atau junior dengan i

In [24]:
df = pd.read_parquet('llama3_70b_social_media_qa-00000-of-00001.parquet')
df.head()

,prompt_input,input,output
0,None,Bru 5hb weyy!! Habess dahh gaji knapa bulan ni...,Terima kasih atas perkongsian anda mengenai ke...
1,None,Inilah penyelesaian masalah Azmin Ali sejak ha...,Terima kasih atas pertanyaan yang dikemukakan....
2,None,Birthday Aku 22/5 apa Aku nak seharian org ant...,Terima kasih atas perkongsian ini! Hari lahir ...
3,None,"PAS yang bertarung seorang diri, hanya mengend...",Terima kasih atas perkongsian maklumat tentang...
4,None,Antara tanda2 kejatuhan sesebuah kerajaan apab...,Tanda-tanda kejatuhan sesebuah kerajaan memang...


In [25]:
social_media = []
for i in range(len(df)):
    d = [
        {'role': 'user', 'content': df.iloc[i]['input']},
        {'role': 'assistant', 'content': df.iloc[i]['output']}
    ]
    social_media.append(d)

In [26]:
social_media[-1]

[{'role': 'user',
  'content': 'cikatilia posted on 22-12-2012 01:35 AM \r\ncantik siot\n\r\njgn ada yg menambah pakai inner sbb bertudung sudahh\nbertudung tu x menawan ke pada ko ?'},
 {'role': 'assistant',
  'content': 'Assalamualaikum cikatilia,\n\nTerima kasih atas komen yang ditinggalkan di ruangan ini. Saya berharap agar kita dapat berbicara dengan santai dan bersifat positif.\n\nBagaimanapun, saya ingin mengetahui lebih lanjut tentang maksud komen "cantik siot" dan "jgn ada yg menambah pakai inner sbb bertudung sudahh". Adakah cikatilia hendak mengatakan bahawa bertudung itu sudah cukup sebagai identiti seorang muslimah dan tidak perlu lagi menambahkan elemen lain untuk menampakkan kecantikan?\n\nSaya cuba memahami perkara ini daripada perspektif yang positif dan santai. Bertudung memang merupakan salah satu elemen penting dalam beragama, namun saya percaya bahawa setiap individu memiliki hak dan kebebasan untuk menentukan pilihan dan gaya mereka sendiri.\n\nJika cikatilia bers

In [27]:
df = pd.read_parquet('llama3_70b_non_bumi_qa-00000-of-00001.parquet')
df.head()

,prompt_input,input,output
0,None,中国新年期间，马来西亚的城市街道为什么会挂满红灯笼和 Couplets？,在马来西亚，城市街道挂满红灯笼和 Couplets 是因为中国新年（也称春节）是马来西亚华人...
1,None,在马来西亚，中国新年被视为一个重要的社会交流和互动的机会，为什么？,在马来西亚，中国新年被视为一个重要的社会交流和互动的机会，因为马来西亚是一个多元种族和文化的...
2,None,马来西亚华人在春节期间通常会举办哪些传统活动，以维护和弘扬中华文化？,在马来西亚，华人在春节期间通常会举办多种传统活动，以维护和弘扬中华文化。以下是一些常见的活动...
3,None,中国新年在马来西亚的经济影响如何？例如，对旅游业、零售业和餐饮业的影响？,中国新年（亦称春节）是马来西亚一个重要的传统节日，对马来西亚的经济产生了显著的影响。以下是对...
4,None,在马来西亚，中国新年期间的食物习俗有什么特点？例如，为什么year cake和鱼生是春节期间...,在马来西亚，中国新年期间的食物习俗具有独特的特点。马来西亚的华人社区.BackgroundI...


In [28]:
nonbumi = []
for i in range(len(df)):
    d = [
        {'role': 'user', 'content': df.iloc[i]['input']},
        {'role': 'assistant', 'content': df.iloc[i]['output']}
    ]
    nonbumi.append(d)

In [29]:
azril = []
df = pd.read_parquet('tatabahasa-00000-of-00001.parquet')
for i in range(len(df)):
    d = [
        {'role': 'user', 'content': df.iloc[i]['input']},
        {'role': 'assistant', 'content': df.iloc[i]['output']}
    ]
    azril.append(d)
    
df = pd.read_parquet('mallm_bench-00000-of-00001.parquet')
for i in range(len(df)):
    d = [
        {'role': 'user', 'content': df.iloc[i]['input']},
        {'role': 'assistant', 'content': df.iloc[i]['output']}
    ]
    azril.append(d)

In [30]:
df = pd.read_parquet('ayat_aktif_pasif-00000-of-00001.parquet')
ayat_aktif = []
for i in range(len(df)):
    d = [
        {'role': 'user', 'content': df.iloc[i]['input']},
        {'role': 'assistant', 'content': df.iloc[i]['output']}
    ]
    ayat_aktif.append(d)
ayat_aktif[0]

[{'role': 'user',
  'content': 'tukar ayat aktif ke ayat pasif: Encik Razak mengajar pelajar-pelajar tentang kepentingan menjaga alam sekitar.'},
 {'role': 'assistant',
  'content': 'Pelajar-pelajar diajar tentang kepentingan menjaga alam sekitar oleh Encik Razak.'}]

In [31]:
df = pd.read_parquet('malaysian_safe_respond-00000-of-00001.parquet')
safe_respond = []
for i in range(len(df)):
    d = [
        {'role': 'user', 'content': df.iloc[i]['input']},
        {'role': 'assistant', 'content': df.iloc[i]['output']}
    ]
    safe_respond.append(d)
safe_respond[0]

[{'role': 'user',
  'content': 'Budak ni makan taik kali bodo sial babi baling mercun kat kote dia'},
 {'role': 'assistant',
  'content': 'Saya tidak dapat memahami sepenuhnya konteks perbualan ini, tetapi saya boleh mengesyorkan bahawa menggunakan bahasa yang kasar dan menghina adalah tidak sesuai dalam sebarang situasi. Adalah penting untuk berkomunikasi dengan hormat dan menghormati orang lain. Jika anda mengalami masalah atau tekanan, saya menggalakkan anda untuk mendapatkan bantuan daripada sumber yang boleh dipercayai seperti keluarga, rakan atau profesional kesihatan mental.'}]

In [32]:
with open('kamus-instructions.json') as fopen:
    kamus_instructions = json.load(fopen)

In [34]:
with open('filter-instructions.json') as fopen:
    filtered_instructions = json.load(fopen)

In [66]:
with open('dialect-instructions.json') as fopen:
    dialect_instructions = json.load(fopen)

In [67]:
with open('jawi-code.json') as fopen:
    force_jawi = json.load(fopen)

In [68]:
with open('manglish-code.json') as fopen:
    force_manglish = json.load(fopen)

In [69]:
gathered = [
    camel_instruct,
    peribahasa_instruct,
    jawi_translation_instruct,
    jawi_instructions,
    code,
    filter_ultrachat,
    social_media,
    nonbumi,
    azril,
    ayat_aktif,
    safe_respond,
]

lists = [
    kamus_instructions,
    dialect_instructions,
    filtered_instructions,
    force_jawi,
    force_manglish,
]

for i in range(len(lists)):
    for v in lists[i].values():
        gathered.append(v)
        
for k in force.keys():
    gathered.append(force[k])

In [70]:
combine = []
errors = []
for n in range(len(gathered)):
    print(n, len(gathered[n]))
    for k in range(len(gathered[n])):
        c = gathered[n][k]
        try:
            success = 0
            for i in range(len(c)):
                if len(c[i]['content']) < 1:
                    break
                c[i]['content'] = c[i]['content'].replace(' Anda ', ' anda ')
                success += 1

            if success == len(c):
                combine.append(c)
        except Exception as e:
            errors.append(c)
            continue
            
len(combine), len(errors)

0 56132
1 8321
2 20000
3 50000
4 114853
5 140432
6 78589
7 8258
8 8097
9 3028
10 2599
11 13514
12 129066
13 18115
14 6040
15 6439
16 86426
17 10753
18 42934
19 42726
20 44128
21 11032
22 44128
23 44128
24 24741
25 16494
26 16494
27 28812
28 14406
29 14406
30 22068
31 22068
32 60000
33 50000
34 11405
35 6000
36 11171
37 7348


(1294946, 164)

In [71]:
with open('combined-malaysian-sft.jsonl', 'w') as fopen:
    for c in combine:
        fopen.write(f'{json.dumps(c)}\n')
        fopen.flush()

In [72]:
from huggingface_hub import HfApi
api = HfApi()

In [73]:
api.upload_file(
    path_or_fileobj="combined-malaysian-sft.jsonl",
    path_in_repo="combine/combined-malaysian-sft.jsonl",
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

combined-malaysian-sft.jsonl:   0%|          | 0.00/5.05G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/504767c02d4ecc12b2fa8266d121b3ec7d6f3c8f', commit_message='Upload combine/combined-malaysian-sft.jsonl with huggingface_hub', commit_description='', oid='504767c02d4ecc12b2fa8266d121b3ec7d6f3c8f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-SFT', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-SFT'), pr_revision=None, pr_num=None)

In [74]:
import copy

gathered = [
    camel_instruct,
    peribahasa_instruct,
    jawi_translation_instruct,
    jawi_instructions,
    code,
    filter_ultrachat,
    social_media,
    nonbumi,
    azril,
    ayat_aktif,
    safe_respond,
]

lists = [
    kamus_instructions,
    dialect_instructions,
    filtered_instructions,
    force_jawi,
    force_manglish,
]
        
for k in force.keys():
    gathered.append(force[k])

combine = []
errors = []
for n in range(len(gathered)):
    print(n, len(gathered[n]))
    g = copy.deepcopy(gathered[n])
    if len(g) > 5000:
        g = random.sample(g, 5000)
    for k in range(len(g)):
        c = g[k]
        try:
            success = 0
            for i in range(len(c)):
                if len(c[i]['content']) < 1:
                    break
                c[i]['content'] = c[i]['content'].replace(' Anda ', ' anda ')
                for r in remove:
                    c[i]['content'] = c[i]['content'].replace(r, '').strip()
                success += 1

            if success == len(c):
                combine.append(c)
        except Exception as e:
            errors.append(c)
            continue
            
for i in range(len(lists)):
    for g in lists[i].values():
        for k in range(len(g)):
            c = g[k]
            try:
                success = 0
                for i in range(len(c)):
                    if len(c[i]['content']) < 1:
                        break
                    c[i]['content'] = c[i]['content'].replace(' Anda ', ' anda ')
                    for r in remove:
                        c[i]['content'] = c[i]['content'].replace(r, '').strip()
                    success += 1

                if success == len(c):
                    combine.append(c)
            except Exception as e:
                errors.append(c)
                continue
            
len(combine), len(errors)

0 56132
1 8321
2 20000
3 50000
4 114853
5 140432
6 78589
7 8258
8 8097
9 3028
10 2599
11 60000
12 50000
13 11405
14 6000
15 11171
16 7348


(739348, 164)

In [75]:
combine[-10000]

[{'role': 'user',
  'content': 'convert,\n```\nHello! The answer to your question is four. Two plus two is a basic arithmetic operation that always equals four. Is there anything else you would like to know about mathematics?\n```\n\nto manglish'},
 {'role': 'assistant',
  'content': 'Wah lao eh! Answer to your question is four lah. Two plus two very simple one, always get four one. You want know anything else about math or not? Can ask me, I tell you.'}]

In [76]:
with open('combined-malaysian-sft-5k-sample.jsonl', 'w') as fopen:
    for c in combine:
        fopen.write(f'{json.dumps(c)}\n')
        fopen.flush()

In [77]:
api.upload_file(
    path_or_fileobj="combined-malaysian-sft-5k-sample.jsonl",
    path_in_repo="combine/combined-malaysian-sft-5k-sample.jsonl",
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

combined-malaysian-sft-5k-sample.jsonl:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/c02461ad08bcda1c54e243f34d3cd6ad90e32a00', commit_message='Upload combine/combined-malaysian-sft-5k-sample.jsonl with huggingface_hub', commit_description='', oid='c02461ad08bcda1c54e243f34d3cd6ad90e32a00', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-SFT', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-SFT'), pr_revision=None, pr_num=None)

In [78]:
import copy

gathered = [
    camel_instruct,
    peribahasa_instruct,
    jawi_translation_instruct,
    jawi_instructions,
    code,
    filter_ultrachat,
    social_media,
    nonbumi,
    azril,
    ayat_aktif,
    safe_respond,
]

lists = [
    kamus_instructions,
    dialect_instructions,
    filtered_instructions,
    force_jawi,
    force_manglish,
]
        
for k in force.keys():
    gathered.append(force[k])

combine = []
errors = []
for n in range(len(gathered)):
    print(n, len(gathered[n]))
    g = copy.deepcopy(gathered[n])
    if len(g) > 10000:
        g = random.sample(g, 10000)
    for k in range(len(g)):
        c = g[k]
        try:
            success = 0
            for i in range(len(c)):
                if len(c[i]['content']) < 1:
                    break
                c[i]['content'] = c[i]['content'].replace(' Anda ', ' anda ')
                for r in remove:
                    c[i]['content'] = c[i]['content'].replace(r, '').strip()
                success += 1

            if success == len(c):
                combine.append(c)
        except Exception as e:
            errors.append(c)
            continue
            
for i in range(len(lists)):
    for g in lists[i].values():
        for k in range(len(g)):
            c = g[k]
            try:
                success = 0
                for i in range(len(c)):
                    if len(c[i]['content']) < 1:
                        break
                    c[i]['content'] = c[i]['content'].replace(' Anda ', ' anda ')
                    for r in remove:
                        c[i]['content'] = c[i]['content'].replace(r, '').strip()
                    success += 1

                if success == len(c):
                    combine.append(c)
            except Exception as e:
                errors.append(c)
                continue
            
len(combine), len(errors)

0 56132
1 8321
2 20000
3 50000
4 114853
5 140432
6 78589
7 8258
8 8097
9 3028
10 2599
11 60000
12 50000
13 11405
14 6000
15 11171
16 7348


(802373, 164)

In [79]:
with open('combined-malaysian-sft-10k-sample.jsonl', 'w') as fopen:
    for c in combine:
        fopen.write(f'{json.dumps(c)}\n')
        fopen.flush()

In [80]:
api.upload_file(
    path_or_fileobj="combined-malaysian-sft-10k-sample.jsonl",
    path_in_repo="combine/combined-malaysian-sft-10k-sample.jsonl",
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

combined-malaysian-sft-10k-sample.jsonl:   0%|          | 0.00/2.34G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/0be17c134a98ef17ad7bcad7969411edc89705c5', commit_message='Upload combine/combined-malaysian-sft-10k-sample.jsonl with huggingface_hub', commit_description='', oid='0be17c134a98ef17ad7bcad7969411edc89705c5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-SFT', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-SFT'), pr_revision=None, pr_num=None)

In [87]:
import copy

gathered = [
    camel_instruct,
    peribahasa_instruct,
    jawi_translation_instruct,
    jawi_instructions,
    code,
    filter_ultrachat,
    social_media,
    nonbumi,
    azril,
    ayat_aktif,
    safe_respond,
]

lists = [
    kamus_instructions,
    dialect_instructions,
    filtered_instructions,
    force_jawi,
    force_manglish,
]
        
for k in force.keys():
    gathered.append(force[k])

combine = []
errors = []
for n in range(len(gathered)):
    print(n, len(gathered[n]))
    g = copy.deepcopy(gathered[n])
    if len(g) > 20000:
        g = random.sample(g, 20000)
    for k in range(len(g)):
        c = g[k]
        try:
            success = 0
            for i in range(len(c)):
                if len(c[i]['content']) < 1:
                    break
                c[i]['content'] = c[i]['content'].replace(' Anda ', ' anda ')
                for r in remove:
                    c[i]['content'] = c[i]['content'].replace(r, '').strip()
                success += 1

            if success == len(c):
                combine.append(c)
        except Exception as e:
            errors.append(c)
            continue
            
for i in range(len(lists)):
    for g in lists[i].values():
        for k in range(len(g)):
            c = g[k]
            try:
                success = 0
                for i in range(len(c)):
                    if len(c[i]['content']) < 1:
                        break
                    c[i]['content'] = c[i]['content'].replace(' Anda ', ' anda ')
                    for r in remove:
                        c[i]['content'] = c[i]['content'].replace(r, '').strip()
                    success += 1

                if success == len(c):
                    combine.append(c)
            except Exception as e:
                errors.append(c)
                continue
            
len(combine), len(errors)

0 56132
1 8321
2 20000
3 50000
4 114853
5 140432
6 78589
7 8258
8 8097
9 3028
10 2599
11 60000
12 50000
13 11405
14 6000
15 11171
16 7348


(884949, 164)

In [88]:
with open('combined-malaysian-sft-20k-sample.jsonl', 'w') as fopen:
    for c in combine:
        fopen.write(f'{json.dumps(c)}\n')
        fopen.flush()

In [ ]:
api.upload_file(
    path_or_fileobj="combined-malaysian-sft-20k-sample.jsonl",
    path_in_repo="combine/combined-malaysian-sft-20k-sample.jsonl",
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

combined-malaysian-sft-20k-sample.jsonl:   0%|          | 0.00/2.85G [00:00<?, ?B/s]

In [84]:
import copy

gathered = [
    camel_instruct,
    peribahasa_instruct,
    jawi_translation_instruct,
    jawi_instructions,
    code,
    filter_ultrachat,
    social_media,
    nonbumi,
    azril,
    ayat_aktif,
    safe_respond,
]

lists = [
    kamus_instructions,
    dialect_instructions,
    filtered_instructions,
    force_jawi,
    force_manglish,
]

for i in range(len(lists)):
    for v in lists[i].values():
        gathered.append(v)
        
for k in force.keys():
    gathered.append(force[k])

combine = []
errors = []
for n in range(len(gathered)):
    print(n, len(gathered[n]))
    g = copy.deepcopy(gathered[n])
    if len(g) > 50:
        g = random.sample(g, 50)
    for k in range(len(g)):
        c = g[k]
        try:
            success = 0
            for i in range(len(c)):
                if len(c[i]['content']) < 1:
                    break
                c[i]['content'] = c[i]['content'].replace(' Anda ', ' anda ')
                success += 1

            if success == len(c):
                combine.append(c)
        except Exception as e:
            errors.append(c)
            continue
            
len(combine), len(errors)

0 56132
1 8321
2 20000
3 50000
4 114853
5 140432
6 78589
7 8258
8 8097
9 3028
10 2599
11 13514
12 129066
13 18115
14 6040
15 6439
16 86426
17 10753
18 42934
19 42726
20 44128
21 11032
22 44128
23 44128
24 24741
25 16494
26 16494
27 28812
28 14406
29 14406
30 22068
31 22068
32 60000
33 50000
34 11405
35 6000
36 11171
37 7348


(1900, 0)

In [85]:
with open('combined-malaysian-sft-50-sample-all.jsonl', 'w') as fopen:
    for c in combine:
        fopen.write(f'{json.dumps(c)}\n')
        fopen.flush()

In [86]:
api.upload_file(
    path_or_fileobj="combined-malaysian-sft-50-sample-all.jsonl",
    path_in_repo="combine/combined-malaysian-sft-50-sample-all.jsonl",
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/c70f2c5409924cc2dc311126280b3ab87c14f84e', commit_message='Upload combine/combined-malaysian-sft-50-sample-all.jsonl with huggingface_hub', commit_description='', oid='c70f2c5409924cc2dc311126280b3ab87c14f84e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-SFT', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-SFT'), pr_revision=None, pr_num=None)